In [ ]:
#from nipype.interfaces import niftyreg
import numpy as np
from skimage import io
from pathlib import Path
import re
import ants
from skimage.transform import resize
from tqdm import tqdm

In [4]:
directory_seg = Path('james_preds')
files_seg  = list(directory_seg.glob('*_seg.npy'))
files_seg = sorted([x.as_posix() for x in files_seg])

In [3]:
for i in tqdm(range(len(files_seg))):
    tmp = np.load(files_seg[i])
    tmp = np.reshape(tmp,(1,507,507,252))
    io.imsave(re.sub('.npy','.tif',files_seg[0]),tmp)

  0%|          | 0/94 [00:00<?, ?it/s]/tmp/ipykernel_2464810/3179607572.py:4: UserWarning:

james_preds/XYZres006_0001_seg.tif is a low contrast image

100%|██████████| 94/94 [00:40<00:00,  2.34it/s]


In [4]:
files_seg  = list(directory_seg.glob('*_seg.tif'))
files_seg = sorted([x.as_posix() for x in files_seg])
files_seg[0]

'james_preds/XYZres006_0001_seg.tif'

In [5]:
image_path = Path('../TBI')
images = list(image_path.glob('vbm*/*?[0-9].tif'))
images = sorted([x.as_posix() for x in images])
len(images)

94

In [6]:
for i in tqdm(range(len(images))):
    image = io.imread(images[i])
    image = np.swapaxes(image,0,1)
    image = np.swapaxes(image,1,3)
    image = resize(image,(2,507,507,252),preserve_range=True)
    zeros = np.zeros((1,507,507,252)).astype('uint16')
    image = np.append(image,zeros,axis = 0)
    image = image.astype('float16')
    io.imsave('james_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name),image)

100%|██████████| 94/94 [39:06<00:00, 24.97s/it]


In [ ]:
#'james_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name

In [5]:
images = list(directory_seg.glob('*_0001_upsamp.tif'))
images = sorted([x.as_posix() for x in images])#[0:2]
images[0]

'james_preds/XYZres006_0001_upsamp.tif'

In [6]:
for i in tqdm(range(len(images))):
    fix = ants.image_read(re.sub('_0001','',images[i]))
    mov = ants.image_read(images[i])
    fix_mask = ants.image_read(re.sub('upsamp','seg',re.sub('_0001','',images[i])))
    mov_mask = ants.image_read(re.sub('upsamp','seg',images[i]))
    mytx = ants.registration(fixed = fix,
                            moving = mov,
                            type_of_transform = 'Rigid'
                            )
    warpedmask = ants.apply_transforms(fixed = fix_mask,
                                       moving = mov_mask,
                                       transformlist = mytx['fwdtransforms'],
                                       interpolator = 'nearestNeighbor'
                                      )
    io.imsave(re.sub('seg','seg_warped',re.sub('upsamp','seg',images[i])),warpedmask.numpy())
    io.imsave(re.sub('seg','seg_warped',re.sub('upsamp','seg',re.sub('_0001','',images[i]))),fix_mask.numpy())
    #print(2 * np.sum(warpedmask.numpy()*fix_mask.numpy())/(np.sum(warpedmask.numpy())+np.sum(fix_mask.numpy())))

  0%|          | 0/47 [00:01<?, ?it/s]


AttributeError: module 'ants.core.ants_image' has no attribute 'ANTsImage'

In [9]:
print(re.sub('seg','seg_warped',re.sub('upsamp','seg',images[i])))

james_preds/XYZres050_0001_seg_warped.tif


In [10]:
io.imsave(re.sub('seg','seg_warped',re.sub('upsamp','seg',images[i])),warpedmask.numpy())